In [1]:
using VCFTool

In [11]:
input_dir = "../input/"

output_dir = "../output/"

vcf_738_file_path = joinpath(input_dir, "738_variants.vcf.gz")

vcf_example_file_path = joinpath(input_dir, "example.vcf.gz")

"../input/example.vcf.gz"

In [6]:
vcf_738 = read_vcf(vcf_738_file_path)

vcf_example = read_vcf(vcf_example_file_path)

vcf_to_use = vcf_example

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
,String,Int64,String,String,String,Int64,String,String
1,1,10,1:10,A,T,100,PASS,NS=5;AC=0;AN=10;AF=0.000000
2,1,20,1:20,G,C,100,PASS,NS=0;AC=0;AN=0
3,1,30,1:30,C,A,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
4,1,40,1:40,A,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
5,1,10000,1:10000,G,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
6,1,20000,1:20000,T,A,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
7,4,5000,4:5000,A,T,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
8,4,6000,4:6000,C,T,100,PASS,NS=6;AC=11;AN=12;AF=0.916667
9,X,800,X:800,A,C,100,PASS,NS=6;AC=12;AN=12;AF=1.000000


## Tabix Regions

In [26]:
bed_file_path = joinpath(input_dir, "cardiotoxicity.bed")

tabix_regions_from_file(
    bed_file_path,
    vcf_738_file_path,
    output_dir,
)

LoadError: UndefVarError: tabix_regions_from_file not defined

## Julia DataFrame Query 

In [6]:
regions = Dict([
        ("chr2", 240630710),
        ("chr7", 150999023),
        ("chr10", 99836239),
        ("chr10", 99851537),
        ("chr16", 88646828),
        ("chr19", 51354484),
        ("chr21", 36146408),
        ])

query_regions(
    regions, 
    vcf_to_use,
    output_dir,
)

chr10
99851537


0×5 DataFrame

chr16
88646828


0×5 DataFrame

chr19
51354484


0×5 DataFrame

chr7
150999023


0×5 DataFrame

chr2
240630710


0×5 DataFrame

chr21
36146408


0×5 DataFrame


Took 2 milliseconds.



## JuliaDB

An unzipped VCF file can be loaded into an indextable or ndsparse array directly using loadtable() or ndsparese() respectively. However, its much faster to query variants if they are in an ndsparse array because the chromosome and position are both part of the index.

In [31]:
using JuliaDB

vcf_table = loadtable(
    joinpath(input_dir, "738_variants.vcf"), 
    delim='\t',
    skiplines_begin=261,
    header_exists=false,
    colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
)

Table with 2846 rows, 10 columns:
Columns:
#   colname  type
───────────────────
1   CHROM    String
2   POS      Int64
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     Int64
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [33]:
using Dates

start_time = now()

for item in vcf_table
    if :POS == 10439
        print("yes")
    end
end

# vcf_table["chr1"][10439, 13284, 13868, 16731]]

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")


Took 40 milliseconds.



In [34]:
vcf_ndsparse = loadndsparse(
    joinpath(input_dir, "738_variants.vcf"),
    delim='\t',
    skiplines_begin=261,
    header_exists=false, 
    colnames=[:CHROM, :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :GERM],
    indexcols=[:CHROM, :POS]
)

2-d NDSparse with 2846 values (8 field named tuples):
    Dimensions#  colname  type
──────────────────
1  CHROM    String
2  POS      Int64
    Values#   colname  type
───────────────────
3   ID       String
4   REF      String
5   ALT      String
6   QUAL     Int64
7   FILTER   String
8   INFO     String
9   FORMAT   String
10  GERM     String

In [36]:
using Dates

start_time = now()

vcf_ndsparse["chr1", [28558]]

end_time = now()

println("\nTook $(canonicalize(Dates.CompoundPeriod(end_time - start_time))).\n")


Took empty period.

